MPC for LTI systems and quadratic cost, with reference tracking

In [7]:
import numpy as np # arrays
import matplotlib.pyplot as plt # plots 
import control as ct # control systems library
import cvxpy as cp
from collections import namedtuple
import time as tt
import rpc_diism.mpc as mpc

Main program

In [ ]:
# Continuous-time plant model
A = np.array([[1, -1],
              [0, 1]])
B = np.array([[0],
              [1]])
C = np.array([[1, 0],
              [0, 1]])
D = np.array([[0],
              [0]])

(no,nx) = C.shape

plant_continuous = ct.ss(A,B,C,D, 
                   inputs='u', states=['x[0]','x[1]'], outputs=['x[0]','x[1]'])

# This is the step to simulate the continuous time plant (and the closed loop)
simulation_dt = 0.01
# Generate a discrete-time simulator of the plant with the simulation step
plant_simulator = ct.c2d(plant_continuous, simulation_dt, 'zoh')

# Controller sampling time
T = 0.1

# Generate discretized plant model for use with the controller
plant_discrete = ct.c2d(plant_continuous, T, 'zoh')

# MPC controller parameter structure field names
p = namedtuple('p', ['ref','A', 'B','C','N','Q','R','F','f','E','e','Xf'])

# Cost weights
p.Q = np.array([[1, 0],[0, 1]])
p.R = np.array([[1]])

# Horizon length
p.N = 20

# Model
p.A = plant_discrete.A
p.B = plant_discrete.B
p.C = C

# State constraints
x_0min = -1.2
x_1min = -1.5 
x_0max = 1
x_1max = 1
p.F = np.array([[1,0],
                [0,1],
                [-1,0],
                [0,-1] ])
p.f = np.array([[x_0max],
                [x_1max],
                [-x_0min],
                [-x_1min] ])

# Input constraints
umin = -12
umax = 1.9
p.E = np.array([[1],
                [-1] ])
p.e = np.array([[umax],
                [-umin] ])

# Constant reference to track (None for no tracking)
p.ref = None

# Plant state and input dimension
(nx,nu) = p.B.shape

# Terminal set type
p.Xf = 'lqr'

# Define the MPC controller system object
controller = mpc.mpc_controller_staticref(p, T, 
                            inputs=['dummyinput','x[0]','x[1]'], outputs='u')

# Create model of controller for simulation with the same integration time
# as the plant
controller_simulator = mpc.sampled_data_controller(controller, simulation_dt)

# Compute closed loop. The closed loop state has nu*p.N states (the controller states)
# plus nx states (the plant simulator states)
closedloop_simulator = ct.interconnect([controller_simulator, plant_simulator], inputs='dummyinput', outputs=['x[0]','x[1]','u'],
                        states = nu*p.N + nx)

# Time vector for simulation
end_time = 6
time = np.arange(0, end_time, simulation_dt)

# Dummy input
dummyinput = np.zeros_like(time)

# The controller state is the optimal input sequence: we initialize it to zero
initu = np.zeros(nu*p.N)

# Initial plant state
initx0 = np.array([-1,1])

# Closed loop initial state
initx = np.hstack((initu, initx0))

# Simulate
start_t = tt.time()
t, resp = ct.input_output_response(closedloop_simulator, time, dummyinput, initx)
stop_t = tt.time()
display("Simulation time:", stop_t - start_t)

# Plot responses
x_0, x_1, u = resp # extract responses
plt.plot(t, x_0, 'b-', label='x_0')
plt.plot(t, x_0min*np.ones_like(t), 'b:', label='x_0min')
plt.plot(t, x_1, 'g-', label='x_1')
plt.plot(t, x_1min*np.ones_like(t), 'g:', label='x_1min')
plt.plot(t, u, 'm-', label='u')
plt.plot(t, umax*np.ones_like(t), 'm:')
plt.plot(t, umin*np.ones_like(t), 'm:', label='umin,umax')
plt.xlabel("Time s")
plt.title("Closed loop response")
plt.legend()

AttributeError: type object 'p' has no attribute 'P'